In [0]:
import os, sys
from ipyparallel import Client
import matplotlib.pyplot as plt
%matplotlib inline
from subprocess import Popen, PIPE
from Bio import SeqIO
import pandas as pd
import pickle
import scandir
import sqlite3
from sqlalchemy import create_engine, MetaData, Table, Column, VARCHAR
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from ipyparallel import Client
import filelock
from pymongo import MongoClient

## Start mongodb on cluster (godel96)

```
numactl --interleave=all \
/gpfs_fs/home/eckertlab/src/mongodb-linux-x86_64-3.2.5/bin/mongod \
--port 27018 \
--dbpath /gpfs_fs/home/eckertlab/Mitra/SWWP_seq
```

In [0]:
mongo = MongoClient('godel96', 27018)
db = mongo.fastq_index

In [0]:
#db.drop_collection("reads") #start this over, if needed

In [0]:
sys.path.append("/home/cfriedline/ipynb/include_utils")

In [0]:
import include_utils as u

In [0]:
root = "/home/cfriedline/eckertlab/Mitra/SWWP_seq"

In [0]:
fastq_files = open(os.path.join(root, "fastq_files.txt")).readlines()
fastq_files = [x.strip() for x in fastq_files]

In [0]:
from Bio.SeqIO.QualityIO import FastqGeneralIterator

In [0]:
rc = Client(profile="sge")
dv = rc[:]
lv = rc.load_balanced_view()

In [0]:
len(dv)

In [0]:
with dv.sync_imports():
    import os
    from sqlalchemy import create_engine, insert
    from Bio.SeqIO.QualityIO import FastqGeneralIterator
    from pymongo import MongoClient

In [0]:
db = mongo.fastq_index
reads = db.reads

In [0]:
# reads.insert_one({})
# reads.index_information()

In [0]:
def process_fastq(f):
    mongo = MongoClient('godel96', 27018)
    db = mongo.fastq_index
    reads = db.reads
    count = 0
    batch = []
    sample = os.path.basename(f)
    for name, seq, qual in FastqGeneralIterator(open(f)):
        batch.append({"read":name.split()[0], "sample":sample})
        count += 1
        if count % 500000 == 0:
            print(sample, count)
            reads.insert_many(batch)
            batch = []
    reads.insert_many(batch)
    return f, count

In [0]:
dv['process_fastq'] = process_fastq

In [0]:
jobs = []
for i, f in enumerate(fastq_files):
    j = lv.apply(process_fastq, f)
    jobs.append(j)

In [0]:
sum([x.ready() for x in jobs])

In [0]:
[x.r for x in jobs if x.ready()]

## Create index on read ID
```
/gpfs_fs/home/eckertlab/src/mongodb-linux-x86_64-3.2.5/bin/mongo --host godel96 --port 27018
> use fastq_index
> db.reads.createIndex({"read":1}, {unique:false})
```